In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# Data Cleaning

In [225]:
df =pd.DataFrame(pd.read_csv('../csv selection/FINAL_clean_sales_dataset.csv',
                             low_memory=False))


In [226]:
# drop useless columns
df.drop(['source', 'facades_number'], axis=1, inplace=True)

In [227]:
# make boolean columns binary
df[['house_is',
    'equipped_kitchen_has',
    'furnished',
    'open_fire',
    'terrace',
    'garden',
    'swimming_pool_has']] = df[['house_is',
                                'equipped_kitchen_has',
                                'furnished',
                                'open_fire',
                                'terrace',
                                'garden',
                                'swimming_pool_has']].astype('int')

In [228]:
# limit cardinality of property subtypes
top_n = 8
top_subtypes = df['property_subtype'].value_counts().index[:top_n]

# reset remaining subtypes to 'other'
df['property_subtype'] = df['property_subtype'].where(
    df['property_subtype'].isin(top_subtypes), other = 'OTHER')
print('Value counts:\n', df['property_subtype'].value_counts())

Value counts:
 APARTMENT             3847
HOUSE                 3358
OTHER                  944
VILLA                  787
APARTMENT_BLOCK        682
MIXED_USE_BUILDING     681
DUPLEX                 390
PENTHOUSE              328
GROUND_FLOOR           271
Name: property_subtype, dtype: int64


In [229]:
def dummy(col, df):
    col_enc = pd.get_dummies(df[col])
    
    df = pd.concat([df, col_enc], axis=1)
    
    df.drop([col], axis=1, inplace=True)
    
    return df

# it is a bit more powerful to write like this:
# drop_first is the dummy encoding, deleting one of the columns per categorical.
# For example, if it is not in Wallonia or Flanders, no column is needed
# saying it is Brussels
def dummy(categoricals, df):
    df = pd.get_dummies(df, columns = categoricals, drop_first = True)
    return df

In [230]:
categoricals = ['property_subtype', 'region', 'building_state_agg']
df = dummy(categoricals, df)
df.columns

Index(['postcode', 'house_is', 'price', 'rooms_number', 'area',
       'equipped_kitchen_has', 'furnished', 'open_fire', 'terrace',
       'terrace_area', 'garden', 'garden_area', 'land_surface',
       'swimming_pool_has', 'property_subtype_APARTMENT_BLOCK',
       'property_subtype_DUPLEX', 'property_subtype_GROUND_FLOOR',
       'property_subtype_HOUSE', 'property_subtype_MIXED_USE_BUILDING',
       'property_subtype_OTHER', 'property_subtype_PENTHOUSE',
       'property_subtype_VILLA', 'region_F', 'region_W',
       'building_state_agg_renovated', 'building_state_agg_to_renovate'],
      dtype='object')

In [231]:
display(df.head())
df.shape

,postcode,house_is,price,rooms_number,area,equipped_kitchen_has,furnished,open_fire,terrace,terrace_area,...,property_subtype_GROUND_FLOOR,property_subtype_HOUSE,property_subtype_MIXED_USE_BUILDING,property_subtype_OTHER,property_subtype_PENTHOUSE,property_subtype_VILLA,region_F,region_W,building_state_agg_renovated,building_state_agg_to_renovate
0,4180,1,295000.0,3.0,242.0,1,0,0,1,36.0,...,0,0,1,0,0,0,0,1,0,0
1,8730,1,675000.0,4.0,349.0,1,0,0,0,0.0,...,0,0,0,0,0,1,1,0,0,0
2,4020,1,250000.0,5.0,303.0,1,0,0,0,0.0,...,0,0,0,0,0,0,0,1,0,1
3,1200,1,545000.0,4.0,235.0,1,1,0,0,0.0,...,0,1,0,0,0,0,0,0,1,0
4,1190,1,500000.0,2.0,220.0,1,0,0,0,0.0,...,0,0,1,0,0,0,0,0,0,0


(11288, 26)

In [232]:
df.drop_duplicates(inplace = True)
df.shape

(11288, 26)

In [233]:
# load zip code table and do necessary translations of towns from french to dutch
zips = pd.read_csv('./zip_prosperity_assets/zipcodes_alpha_nl.csv')
zips['Hoofdgemeente'] = zips['Hoofdgemeente'].replace({
    'TOURNAI': 'DOORNIK',
    'NAMUR': 'NAMEN',
    'MONS': 'BERGEN',
    'ATH': 'AAT',
    'HANNUT': 'HANNUIT',
    'LIÈGE': 'LUIK',
    'JODOIGNE': 'GELDENAKEN',
    'BEVEREN-WAAS': 'BEVEREN (SINT-NIKLAAS)',
    'AALST': 'AALST (AALST)',
    'LESSINES': 'LESSEN',
    'GENAPPE': 'GENEPIËN',
    'SILLY': 'OPZULLIK',
    'WAREMME': 'BORGWORM',
    'SOIGNIES': 'ZINNIK',
    'BLÉGNY': 'BLEGNY',
    "BRAINE-L'ALLEUD": 'EIGENBRAKEL',
    'HUY': 'HOEI',
    'VORST': 'VORST (BRUSSEL-HOOFDSTAD)',
    'ARLON': 'AARLEN',
    'NIVELLES': 'NIJVEL',
    'WAVRE': 'WAVER',
    'HALLE': 'HALLE (HALLE-VILVOORDE)',
    'BASTOGNE': 'BASTENAKEN',
    'SAINT-NICOLAS': 'SAINT-NICOLAS (LUIK)',
    'GREZ-DOICEAU': 'GRAVEN',
    'SINT-JOOST-TEN-NOODE': 'SINT-JOOST-TEN-NODE',
    "BRAINE-LE-COMTE": "'S GRAVENBRAKEL"
})

# load prosperity index table
prosp = pd.read_csv('./zip_prosperity_assets/town_prosperity.csv')
prosp['Administratieve Eenheid'] = prosp['Administratieve Eenheid'].str.upper()
display(zips[['Postcode', 'Hoofdgemeente']])
display(prosp)

# create prosperity lookup table
prosperity = pd.merge(zips, prosp, left_on = 'Hoofdgemeente',
                      right_on = 'Administratieve Eenheid')
prosperity = prosperity.drop_duplicates('Postcode')
prosperity = prosperity[['Postcode', 'Hoofdgemeente', 'Welvaartsindex']]
prosperity


,Postcode,Hoofdgemeente
0,2970,SCHILDE
1,3700,TONGEREN
2,7510,3 Suisses
3,9420,ERPE-MERE
4,8511,KORTRIJK
...,...,...
2820,9630,ZWALM
2821,8550,ZWEVEGEM
2822,8750,WINGENE
2823,9052,GENT


,Administratieve Eenheid,Welvaartsindex
0,NaN,NaN
1,BELGIË,100.0
2,VLAAMS GEWEST,107.0
3,WAALS GEWEST,94.0
4,DUITSTALIGE GEMEENSCHAP,97.0
...,...,...
635,DOISCHE,86.0
636,FLORENNES,92.0
637,PHILIPPEVILLE,91.0
638,WALCOURT,101.0


,Postcode,Hoofdgemeente,Welvaartsindex
0,2970,SCHILDE,144.0
2,3700,TONGEREN,101.0
21,9420,ERPE-MERE,116.0
30,8511,KORTRIJK,105.0
31,8510,KORTRIJK,105.0
...,...,...,...
2664,2160,WOMMELGEM,112.0
2665,9240,ZELE,98.0
2666,9060,ZELZATE,98.0
2667,3520,ZONHOVEN,107.0


In [234]:
# show postcodes that don't resolve in a prosperity index, for
# improving the lookup table
na_mask = pd.merge(df,
                   prosperity,
                   left_on = 'postcode',
                   right_on = 'Postcode',
                   how = 'left').drop(
    ['Postcode', 'Hoofdgemeente'], axis = 1).Welvaartsindex.isna()
df.loc[na_mask, 'postcode'].value_counts()

7090    28
9100    28
6840    22
1480    20
8700    15
1440    15
2950    12
1310    12
6140    11
1360     9
7050     8
4600     8
6141     8
1830     7
4690     7
1460     6
6747     6
4287     5
1320     5
7890     4
6120     4
4360     3
4830     3
7760     3
9220     3
9180     2
4602     2
7750     2
1831     1
9111     1
4750     1
2540     1
Name: postcode, dtype: int64

In [235]:
# merge prosperity into the dataset
df = pd.merge(df,
              prosperity,
              left_on = 'postcode',
              right_on = 'Postcode').drop(
    ['Postcode', 'Hoofdgemeente'], axis = 1)

# position prosperity_index in stead of postcode
pop = list(df.columns).pop()
column_list = [pop] + list(df.columns)[:-1]
df = df.reindex(columns=column_list)
df.drop('postcode', axis = 1, inplace = True)
df.rename(columns={"Welvaartsindex": "prosperity_index"}, inplace = True)

display(df.head())
df.shape

,prosperity_index,house_is,price,rooms_number,area,equipped_kitchen_has,furnished,open_fire,terrace,terrace_area,...,property_subtype_GROUND_FLOOR,property_subtype_HOUSE,property_subtype_MIXED_USE_BUILDING,property_subtype_OTHER,property_subtype_PENTHOUSE,property_subtype_VILLA,region_F,region_W,building_state_agg_renovated,building_state_agg_to_renovate
0,95.0,1,295000.0,3.0,242.0,1,0,0,1,36.0,...,0,0,1,0,0,0,0,1,0,0
1,95.0,1,229000.0,4.0,312.0,1,0,0,1,20.0,...,0,0,1,0,0,0,0,1,0,1
2,95.0,1,150000.0,6.0,450.0,0,0,0,1,29.0,...,0,0,0,0,0,0,0,1,0,1
3,95.0,1,195000.0,3.0,242.0,1,0,0,1,36.0,...,0,1,0,0,0,0,0,1,0,0
4,95.0,1,419000.0,6.0,192.0,1,0,0,1,50.0,...,0,0,0,0,0,1,0,1,0,0


(11026, 26)

In [237]:
df.to_csv("Data_for_Regression.csv", index=False)

# Data Inspection

In [144]:
df.shape

(11288, 43)

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11288 entries, 0 to 11287
Data columns (total 43 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   postcode              11288 non-null  int64  
 1   house_is              11288 non-null  int64  
 2   price                 11288 non-null  float64
 3   rooms_number          11288 non-null  float64
 4   area                  11288 non-null  float64
 5   equipped_kitchen_has  11288 non-null  int64  
 6   furnished             11288 non-null  int64  
 7   open_fire             11288 non-null  int64  
 8   terrace               11288 non-null  int64  
 9   terrace_area          11288 non-null  float64
 10  garden                11288 non-null  int64  
 11  garden_area           11288 non-null  float64
 12  land_surface          11288 non-null  float64
 13  swimming_pool_has     11288 non-null  int64  
 14  APARTMENT             11288 non-null  uint8  
 15  APARTMENT_BLOCK    

In [146]:
df.describe()

,postcode,house_is,price,rooms_number,area,equipped_kitchen_has,furnished,open_fire,terrace,terrace_area,...,SERVICE_FLAT,TOWN_HOUSE,TRIPLEX,VILLA,B,F,W,good,renovated,to_renovate
count,11288.000000,11288.000000,1.128800e+04,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,...,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000,11288.000000
mean,4857.760454,0.537562,4.927968e+05,3.199947,195.134036,0.836641,0.037296,0.064670,0.671775,12.608965,...,0.006733,0.006467,0.002481,0.069720,0.193746,0.494773,0.311481,0.752303,0.072023,0.175673
std,3181.093932,0.498609,5.676955e+05,3.786751,219.598588,0.369710,0.189495,0.245954,0.469588,28.274838,...,0.081781,0.080161,0.049745,0.254686,0.395249,0.499995,0.463119,0.431694,0.258538,0.380559
min,1000.000000,0.000000,2.500000e+04,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1420.000000,0.000000,2.250000e+05,2.000000,97.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,4420.000000,1.000000,3.290000e+05,3.000000,146.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,8300.000000,1.000000,5.300000e+05,4.000000,227.000000,1.000000,0.000000,0.000000,1.000000,15.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,9992.000000,1.000000,1.500000e+07,204.000000,9250.000000,1.000000,1.000000,1.000000,1.000000,708.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [147]:
# Checking Null values
df.isnull().sum()*100/df.shape[0]
# There are no NULL values in the dataset, hence it is clean.

postcode                0.0
house_is                0.0
price                   0.0
rooms_number            0.0
area                    0.0
equipped_kitchen_has    0.0
furnished               0.0
open_fire               0.0
terrace                 0.0
terrace_area            0.0
garden                  0.0
garden_area             0.0
land_surface            0.0
swimming_pool_has       0.0
APARTMENT               0.0
APARTMENT_BLOCK         0.0
BUNGALOW                0.0
CASTLE                  0.0
CHALET                  0.0
COUNTRY_COTTAGE         0.0
DUPLEX                  0.0
EXCEPTIONAL_PROPERTY    0.0
FARMHOUSE               0.0
FLAT_STUDIO             0.0
GROUND_FLOOR            0.0
HOUSE                   0.0
KOT                     0.0
LOFT                    0.0
MANOR_HOUSE             0.0
MANSION                 0.0
MIXED_USE_BUILDING      0.0
OTHER_PROPERTY          0.0
PENTHOUSE               0.0
SERVICE_FLAT            0.0
TOWN_HOUSE              0.0
TRIPLEX             